<a href="https://colab.research.google.com/github/yu-xiu/2020-Fall-CS-160-Team-Lake-Laogai/blob/main-branch/DataSci_Group_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stanford University Summer 2024
## DataSci 112: Principles of Data Science Group Project
### Average Resting Heart Rate Prediction During Sleep Using Fitbit Data
### TEAM: Panda Bytes
**Members:_______Melis Erva Cakir, Yusuf Sina Ovali, Yu Xiu____**

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, f1_score

## Loading Data

In [ ]:
# loading data from google drive
daily_activity_csv = "/content/data/daily_activity_df.csv"
df_da = pd.read_csv(daily_activity_csv)
df_da.head()

,Unnamed: 0,Id,Date,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,TotalMinutes,TotalHours,DayOfWeek,DailyGoalMet,GoalStreak,TotalActiveMinutes,SedentaryRatio,Weekend
0,0,1503960366,2016-03-25,11004,33,12,205,804,1819,1054,18.0,Friday,1,1,250,0.762808,0
1,1,1503960366,2016-03-26,17609,89,17,274,588,2154,968,16.0,Saturday,1,2,380,0.607438,1
2,2,1503960366,2016-03-27,12736,56,5,268,605,1944,934,16.0,Sunday,1,3,329,0.647752,1
3,3,1503960366,2016-03-28,13231,39,20,224,1080,1932,1363,23.0,Monday,1,4,283,0.792370,0
4,4,1503960366,2016-03-29,12041,28,28,243,763,1886,1062,18.0,Tuesday,1,5,299,0.718456,0


In [ ]:
resting_heart_rate_csv = "/content/data/resting_heart_rate.csv"
df_rhr = pd.read_csv(resting_heart_rate_csv)
df_rhr

,Unnamed: 0,ID,Start_Date,Mean_RHR,Median_RHR,Std_Dev_RHR,Lowest_RHR,Max_RHR,Total_Minutes,Coefficient_of_Variation
0,0,5553957443,2016-04-01,68.6,69.0,2.8,57,89,259,4.1
1,1,5553957443,2016-04-02,64.0,64.0,2.9,56,80,214,4.6
2,2,5553957443,2016-04-03,57.2,57.0,3.3,50,78,191,5.7
3,3,5553957443,2016-04-04,58.2,58.0,2.7,52,73,258,4.6
4,4,5553957443,2016-04-05,60.7,61.0,3.1,54,111,268,5.2
...,...,...,...,...,...,...,...,...,...,...
59,59,6962181067,2016-04-10,71.2,71.0,3.8,64,89,191,5.3
60,60,6962181067,2016-04-11,65.5,65.0,2.6,58,75,300,4.0
61,61,6962181067,2016-04-12,68.8,69.0,3.8,60,96,278,5.5
62,62,6775888955,2016-04-02,89.5,91.0,6.3,71,112,213,7.1


In [ ]:
sleep_feat_csv = "/content/data/sleep_feat.csv"
df_sf = pd.read_csv(sleep_feat_csv)
df_sf

,Unnamed: 0,ID,date,wake_duration_minutes,sleep_onset,restless_duration_minutes,sleep_duration_minutes,wakeup_time,bedtime,total_time_in_bed_minutes,restless_percentage,sleep_percentage,wake_percentage,total_sleep_duration_minutes,waso,midpoint,sleep_efficiency,sleep_interruptions,arousal_index
0,0,1927972279,2016-03-13,1.0,2016-03-13 23:40:00,5.0,193.0,2016-03-14 07:49:00,2016-03-13 23:40:00,199.0,2.5,97.0,0.5,198.0,1.0,2016-03-14 03:44:30,99.48,1.0,0.3
1,1,1927972279,2016-03-17,4.0,2016-03-17 23:54:00,21.0,505.0,2016-03-18 10:14:00,2016-03-17 23:54:00,530.0,4.0,95.3,0.8,526.0,4.0,2016-03-18 05:04:00,99.21,4.0,0.5
2,2,1927972279,2016-03-20,0.0,2016-03-20 22:49:30,20.0,403.0,2016-03-21 05:52:30,2016-03-20 22:49:30,423.0,4.7,95.3,NaN,423.0,0.0,2016-03-21 02:21:00,100.00,0.0,0.0
3,3,1927972279,2016-03-23,3.0,2016-03-23 22:33:00,16.0,505.0,2016-03-24 08:27:00,2016-03-23 22:33:00,524.0,3.1,96.4,0.6,521.0,3.0,2016-03-24 03:30:00,99.41,3.0,0.3
4,4,1927972279,2016-03-31,3.0,2016-03-31 23:37:00,19.0,384.0,2016-04-01 06:23:00,2016-03-31 23:37:00,406.0,4.7,94.6,0.7,403.0,3.0,2016-04-01 03:00:00,99.22,3.0,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,92,4702921684,2016-03-28,3.0,2016-03-28 23:08:30,14.0,395.0,2016-03-29 06:00:30,2016-03-28 23:08:30,412.0,3.4,95.9,0.7,409.0,3.0,2016-03-29 02:34:30,99.24,3.0,0.4
93,93,4702921684,2016-03-29,9.0,2016-03-29 22:50:30,18.0,405.0,2016-03-30 06:02:30,2016-03-29 22:50:30,432.0,4.2,93.8,2.1,423.0,9.0,2016-03-30 02:26:30,97.78,9.0,1.3
94,94,4702921684,2016-03-31,0.0,2016-03-31 22:59:00,17.0,398.0,2016-04-01 05:54:00,2016-03-31 22:59:00,415.0,4.1,95.9,NaN,415.0,0.0,2016-04-01 02:26:30,100.00,0.0,0.0
95,95,4702921684,2016-04-03,0.0,2016-04-03 22:02:30,36.0,445.0,2016-04-04 06:03:30,2016-04-03 22:02:30,481.0,7.5,92.5,NaN,481.0,0.0,2016-04-04 02:03:00,100.00,0.0,0.0


## Helper Functions

In [ ]:
def split_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = .9,
                                              shuffle = True, random_state=42)
    return X_train, y_train, X_test, y_test

In [ ]:
def train_model(model, X_train, y_train, X_test, y_test):
    pipeline = make_pipeline(StandardScaler(), model)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    return pipeline, y_pred

In [ ]:
def get_MAE(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    return mae

In [ ]:
def get_MSE(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    return mse

In [ ]:
def get_RMSE(mse):
    rmse = np.sqrt(mse)
    return rmse

In [ ]:
def get_f1_score(y_test, y_pred):
    f1 = f1_score(y_test, y_pred)
    return f1

In [ ]:
def compute_cv_scores(model, X, y, scoring, cv):
    scores = cross_val_score(
      model,
      X=X,
      y=y,
      scoring=scoring,
      cv=cv)
    return scores

In [ ]:
def negate_MSE(scores):
    return -scores.mean()

In [ ]:
def sqrt_negated_MSE(scores):
    return np.sqrt(-scores.mean())

## Splitting data

In [ ]:
## TODO define training and target data
X = "place_holder"
y = "place_holder"

In [ ]:
X_train, y_train, X_test, y_test = split_data(X, y)

## K Nearest Neighbors (KNN) model


In [ ]:
knn = KNeighborsRegressor(n_neighbors=5)
pipeline_knn, y_pred_knn = train_model()

### Cross Validation

### Hyperparameter Tuning

## Linear Regressor model

### Cross Validation

### Hyperparameter Tuning

## Comparison